In [1]:
import os
import sys

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

sys.path.append('../../src')
from nnclib.utils import reshape_weights


def get_raw_epsilons(model_name, base=os.path.expanduser("~/tmp/nnc_weights")):
    """TODO(vatai) proper documentation - just notes now.
    
    Args:
        model_name (str): Search for files starting with
            `model_name` in `base`.

        base (str): A path to the directory to search for 
            numpy arrays which are the weights.
        
    Returns:
        result (dict): A dictionary with a single key which 
            is `model_name` and a matrix of "epsilons". 
    """
    from glob import glob
    from os.path import join


    pattern = join(base, model_name) + "*"
    file_list = glob(pattern)

    data = {}
    for file_name in file_list:        
        weights = np.load(file_name)
        norms = np.linalg.norm(weights, axis=0)
        sorted_weights = np.sort(weights, axis=0) / norms
        # plt.plot(sorted_weights)
        # plt.show()

        mean = np.mean(sorted_weights,
                       axis=1)[:, np.newaxis]
        layer_epsilons = np.abs(sorted_weights - mean)
        key = os.path.basename(file_name)
        data[key] = layer_epsilons
        # plt.plot(layer_epsilons)
        # plt.show()
    return data


def proc(weights, funs):
    """See next function. """
    funs = list(reversed(funs))
    n = len(funs)
    for i, f in enumerate(funs):
        if n == 3 and i == 0:
            weights = list(map(lambda w: f(w, axis=0), weights))
        elif n >= 2:
            weights = map(f, weights)
        if i == n-1:
            weights = list(map(np.ravel, weights))
            weights = np.concatenate(weights)
            weights = f(weights)
    return weights


def proca(weights, funs):
    """See next function. (Yes one more down.)"""
    funs = funs.copy()
    if len(funs) == 3:
        f = funs.pop()
        weights = list(map(lambda w: f(w, axis=0), weights))
    if len(funs) == 2:
        f = funs.pop()
        weights = list(map(f, weights))
    f = funs.pop()
    weights = list(map(np.ravel, weights))
    weights = np.concatenate(weights)
    return f(weights)


def apply_funs_to_weigths(weights, funs):
    """Apply `funs` (functions) to `weights`.
    Note: the functions are applied from last to first.

    Args:
        weights (list): List of numpy matrices.
        
        funs (list): list of (numpy) functions with 
            `len(funs)` equal 2 or 3.
    Returns:
        float: The result of unctions applied to the 
            weights in revers order.
    
    If `funs = [np.min, np.average, np.max], then the 
    return vaule is the minimum of the average of the 
    maximum of weights of columns of the weights.
    """
    n = len(funs)
    if n == 3:
        f = lambda w: funs[2](w, axis=0)
        weights = list(map(f, weights))
    if n >= 2:
        weights = list(map(funs[1], weights))
    weights = list(map(np.ravel, weights))
    weights = np.concatenate(weights)
    return funs[0](weights)


def procc()

data = get_raw_epsilons('xception')


In [2]:
import itertools

test_funs = [np.max, np.min, np.average, np.median]

for r in range(1, 4):
    for comb in itertools.product(test_funs, repeat=r):
        res = proc(data.values(), list(comb))
        resa = proca(data.values(), list(comb))
        resb = procb(data.values(), list(comb))
        if res != resa or res != resb or resa != resb:
            print('Error')
            print(res, resa, resb, list(comb))
        else:
            print('*', end='')
print('done')

************************************************************************************done
